In [ ]:
#pip install yt-dlp

In [2]:
from concurrent.futures import ThreadPoolExecutor
import yt_dlp
import time
import os
from urllib.parse import urlparse

# wget 사용으로 메시지 출력
def check_wget_installed():
    if os.system("wget --version") != 0:
        print("[ERROR] wget이 설치되어 있지 않습니다. 설치 후 다시 시도해주세요.")
        exit(1)

# 형식 목록 확인
def list_formats(video_url):
    try:
        with yt_dlp.YoutubeDL({'listformats': True}) as ydl:
            info = ydl.extract_info(video_url, download=False)
            print("\n[INFO] 사용 가능한 형식:")
            for fmt in info['formats']:
                print(f"ID: {fmt['format_id']}, Resolution: {fmt['resolution']}, Size: {fmt.get('filesize', 'Unknown')}")
    except Exception as e:
        print(f"[ERROR] 형식 목록을 가져오는 중 오류 발생: {e}")

# 동영상 다운로드 함수
def download_video(video_url, output_filename):
    ydl_opts = {
        'format': 'bestvideo[filesize<=50M]+bestaudio/best',  # 크기 제한 완화
        'outtmpl': f'{output_filename}.%(ext)s',  # 파일 이름 템플릿
        'quiet': False,  # 메시지를 출력
        'external_downloader': 'wget',  # wget 사용
        'external_downloader_args': ['--limit-rate=1M'],  # wget 속도 제한 (옵션)
        'noprogress': False,  # 진행 표시 활성화
        'writesubtitles': True,  # 자막 다운로드 활성화
        'writeautomaticsub': True,  # 자동 생성 자막 다운로드 활성화
        'subtitleslangs': ['en', 'ko'],  # 다운로드할 자막 언어 (영어와 한국어)
        'subtitlesformat': 'vtt',  # 자막 파일 형식 (VTT)
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
            print(f"[INFO] 다운로드 및 자막 완료: {output_filename}")
    except Exception as e:
        print(f"[ERROR] 다운로드 실패: {video_url}, 오류: {e}")
        return False

    return True

# 자막 파일 저장 경로 확인 및 출력
def print_subtitle_location(output_filename):
    subtitle_files = [
        f"{output_filename}.en.vtt",
        f"{output_filename}.ko.vtt"
    ]
    for file in subtitle_files:
        if os.path.exists(file):
            print(f"[INFO] 자막 파일 저장 위치: {os.path.abspath(file)}")
        else:
            print(f"[WARNING] 자막 파일을 찾을 수 없습니다: {file}")

# 병렬 처리 함수
def download_videos_concurrently(video_urls):
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:  # 최대 10개의 작업 동시 실행
        futures = []
        for idx, video_url in enumerate(video_urls):
            parsed_url = urlparse(video_url)
            output_filename = parsed_url.path.strip("/").replace("/", "_") or f"Video_{idx + 1}"
            futures.append(executor.submit(download_video, video_url, output_filename))
        
        for idx, future in enumerate(futures):
            result = future.result()
            results.append((video_urls[idx], result))
    return results

# 메인 실행
if __name__ == "__main__":
    # wget 설치 여부 확인
    check_wget_installed()

    # 다운로드할 동영상 URL 목록
    video_urls = [
        "#https://www.ces.tech/videos/2025/january/ces-2025-enterprise-evolution/",
        # 필요한 다른 URL 추가
    ]

    # 형식 목록 확인
    for url in video_urls:
        list_formats(url)

    start_time = time.time()
    results = download_videos_concurrently(video_urls)

    # 결과 요약 출력
    print("\n[INFO] 다운로드 결과 요약:")
    for idx, (url, success) in enumerate(results):
        status = "성공" if success else "실패"
        print(f"  {idx + 1}. {url} -> {status}")

        if success:
            output_filename = urlparse(url).path.strip("/").replace("/", "_") or f"Video_{idx + 1}"
            print_subtitle_location(output_filename)

    print(f"\n[INFO] 모든 동영상 다운로드 완료! 소요 시간: {time.time() - start_time:.2f}초")


[ERROR] wget이 설치되어 있지 않습니다. 설치 후 다시 시도해주세요.
[generic] Extracting URL: https://www.youtube.com/watch/XVo--k1JCrA
[generic] XVo--k1JCrA: Downloading webpage


[generic] XVo--k1JCrA: Extracting information
[youtube] Extracting URL: https://www.youtube.com/embed/XVo--k1JCrA
[youtube] XVo--k1JCrA: Downloading webpage
[youtube] XVo--k1JCrA: Downloading ios player API JSON
[youtube] XVo--k1JCrA: Downloading tv player API JSON
[youtube] XVo--k1JCrA: Downloading m3u8 information
[info] Available formats for XVo--k1JCrA:
ID    EXT   RESOLUTION FPS CH |   FILESIZE   TBR PROTO | VCODEC          VBR ACODEC      ABR ASR MORE INFO
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
sb3   mhtml 48x27        1    |                  mhtml | images                                  storyboard
sb2   mhtml 89x45        1    |                  mhtml | images                                  storyboard
sb1   mhtml 179x90       1    |                  mhtml | images                                  storyboard
sb0   mhtml 359x180      1    |           

[generic] XVo--k1JCrA: Extracting information
[youtube] Extracting URL: https://www.youtube.com/embed/XVo--k1JCrA
[youtube] XVo--k1JCrA: Downloading webpage
[youtube] XVo--k1JCrA: Downloading ios player API JSON
[youtube] XVo--k1JCrA: Downloading tv player API JSON
[youtube] XVo--k1JCrA: Downloading m3u8 information
[info] XVo--k1JCrA: Downloading subtitles: en, ko
[info] XVo--k1JCrA: Downloading 1 format(s): 399+251-5
[info] Writing video subtitles to: watch_XVo--k1JCrA.en.vtt
[download] Destination: watch_XVo--k1JCrA.en.vtt
[download] 100% of   21.57KiB in 00:00:00 at 144.84KiB/s
[info] Writing video subtitles to: watch_XVo--k1JCrA.ko.vtt
[download] Destination: watch_XVo--k1JCrA.ko.vtt
[download] 100% of   25.38KiB in 00:00:00 at 118.95KiB/s


ERROR: You have requested merging of multiple formats but ffmpeg is not installed. Aborting due to --abort-on-error


[ERROR] 다운로드 실패: https://www.youtube.com/watch/XVo--k1JCrA, 오류: ERROR: You have requested merging of multiple formats but ffmpeg is not installed. Aborting due to --abort-on-error

[INFO] 다운로드 결과 요약:
  1. https://www.youtube.com/watch/XVo--k1JCrA -> 실패

[INFO] 모든 동영상 다운로드 완료! 소요 시간: 4.84초
